In [1]:
from Dataset_importing import load_housing_data
from sklearn.model_selection import train_test_split

Now, we will import that data set divide it into two parts training & test set.

In [2]:
import pandas as pd
import numpy as np
housing=load_housing_data()
housing["income_cat"]=pd.cut(housing["median_income"],bins=[0,1.5,3,4.5,6,np.inf],labels=[1,2,3,4,5])
train_set,test_set=train_test_split(housing,stratify=housing["income_cat"],test_size=0.2,random_state=42)
for set_ in (train_set,test_set):
    set_.drop(["income_cat"],inplace=True,axis=1)
train_set.describe()
housing=train_set.drop("median_house_value",axis=1)
housing_labels=train_set["median_house_value"].copy()

The next step is to create functions for performing the following tasks:
1. For creating ratios,
2. For making a clustering Transformer
3. Making Pipelines

In [3]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel

The ratio_name function has to have two arguments because this is how the feature_names_out parameter of scikitlearn FunctionTransformer expects them so def ratio_name(__,___) will also work if you don't want to later use these arguments.

In [4]:
def ratio(X):
    return X[:,[0]]/X[:,[1]]
def ratio_name(function_Transformer,feature_names_in):
    return ["ratio"]
def ratio_pipeline():
    return make_pipeline(SimpleImputer(strategy="median"),FunctionTransformer(ratio,feature_names_out=ratio_name,),StandardScaler())

Now, we move on to creating a class for Clustering algorithm. This clustering algorith creates random clusters than groups them according to their cluster centers which are choosen randomly, then rbf kernel finds the similarity between these cluster centers.


In [5]:
class Cluster(BaseEstimator,TransformerMixin):
    def __init__(self,random_state=None,gamma=1,n_clusters=10):
        self.n_clusters=n_clusters
        self.gamma=gamma
        self.random_state=random_state
    
    def fit(self,X,y=None,sample_weight=None):
        self.K=KMeans(self.n_clusters,n_init=10,random_state=self.random_state)
        self.K.fit(X,sample_weight=sample_weight)
        return self
    
    def transform(self,X):
        return rbf_kernel(X,self.K.cluster_centers_,gamma=self.gamma)
    def get_feature_names_out(self,names=None):
        return [f"Clusters {i} similarity" for i in range (self.n_clusters)]


Now we can start creating our pipelines:
1. Logrithmic Pipeline
2. Cat_pipeline
3. Clustering Pipeline (maybe not required)

In [6]:
from sklearn.preprocessing import OneHotEncoder
log_pipeline=make_pipeline(SimpleImputer(strategy="median"),FunctionTransformer(np.log,feature_names_out="one-to-one"),StandardScaler())
cat_pipeline=make_pipeline(SimpleImputer(strategy="most_frequent"),OneHotEncoder())
default_num_pipeline=make_pipeline(SimpleImputer(strategy="median"),StandardScaler())


In [7]:
Cluster_similarity=Cluster(random_state=42,gamma=1,n_clusters=10)
housing.describe()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
count,16512.000000,16512.000000,16512.000000,16512.000000,16344.000000,16512.000000,16512.000000,16512.000000
mean,-119.573125,35.637746,28.577156,2639.402798,538.949094,1425.513929,499.990189,3.870428
std,2.000624,2.133294,12.585738,2185.287466,423.862079,1094.795467,382.865787,1.891936
min,-124.350000,32.550000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900
25%,-121.800000,33.930000,18.000000,1447.000000,296.000000,787.000000,279.000000,2.562500
50%,-118.510000,34.260000,29.000000,2125.000000,434.000000,1167.000000,408.000000,3.538500
75%,-118.010000,37.720000,37.000000,3154.000000,645.000000,1726.000000,603.000000,4.750000
max,-114.490000,41.950000,52.000000,39320.000000,6210.000000,16305.000000,5358.000000,15.000100


Let's make the column selector:

It does not matter if we pass SimpleImputer in case of clusters or not but just to make sure everything remains nice

In [8]:
preprocessing=ColumnTransformer([("bedrooms",ratio_pipeline(),["total_bedrooms","total_rooms"]),
                                 ("People_per_house",ratio_pipeline(),["population","households"]),
                                 ("rooms_per_house",ratio_pipeline(),["total_rooms","households"]),
                                 ("log",log_pipeline,["total_rooms","total_bedrooms","population","households","median_income"]),
                                 ("category",cat_pipeline,make_column_selector(dtype_include=object)),
                                 ("geo",Cluster_similarity,["longitude","latitude"])],remainder=default_num_pipeline)
housing_tranformed=preprocessing.fit_transform(housing)
housing_tranformed.shape
preprocessing.get_feature_names_out()

array(['bedrooms__ratio', 'People_per_house__ratio',
       'rooms_per_house__ratio', 'log__total_rooms',
       'log__total_bedrooms', 'log__population', 'log__households',
       'log__median_income', 'category__ocean_proximity_<1H OCEAN',
       'category__ocean_proximity_INLAND',
       'category__ocean_proximity_ISLAND',
       'category__ocean_proximity_NEAR BAY',
       'category__ocean_proximity_NEAR OCEAN',
       'geo__Clusters 0 similarity', 'geo__Clusters 1 similarity',
       'geo__Clusters 2 similarity', 'geo__Clusters 3 similarity',
       'geo__Clusters 4 similarity', 'geo__Clusters 5 similarity',
       'geo__Clusters 6 similarity', 'geo__Clusters 7 similarity',
       'geo__Clusters 8 similarity', 'geo__Clusters 9 similarity',
       'remainder__housing_median_age'], dtype=object)

Now, we will move towards testing different models/algorithms


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
model=LinearRegression()
Lin_Pipeline=make_pipeline(preprocessing,model)
Lin_Pipeline.fit(housing,housing_labels)
Lin_predictions=Lin_Pipeline.predict(housing)
Lin_predictions[:5].round(-2)


array([242800., 375900., 127500.,  99400., 324600.])

Comparing against the actual housing_labels which is a dataframe; hence the iloc

In [10]:
housing_labels.iloc[:5].values

array([458300., 483800., 101700.,  96100., 361800.])

In [11]:
from sklearn.metrics import root_mean_squared_error
Lin_error=root_mean_squared_error(Lin_predictions,housing_labels)
Lin_error

68647.95686706687

Now, we will be verifying the cross_validation; it basically uses mutliple sets.

In [12]:
from sklearn.model_selection import cross_val_score
Lin_validation=-cross_val_score(Lin_Pipeline,housing,housing_labels,scoring="neg_root_mean_squared_error",cv=10)
pd.Series(Lin_validation).describe()

count       10.000000
mean     69847.923224
std       4078.407329
min      65659.761079
25%      68088.799156
50%      68697.591463
75%      69800.966364
max      80685.254832
dtype: float64

We need to utilise a new and better model, the one that I found HisGradientBoostingRegressor.

In [13]:
model_2=HistGradientBoostingRegressor()
Hist_Reg=make_pipeline(preprocessing,model_2)
Hist_Reg.fit(housing,housing_labels)
Hist_prediction=Hist_Reg.predict(housing)
Hist_prediction[:5].round(-2)


array([391800., 444600., 102500.,  95800., 369200.])

Now, we will compare these and find the difference.

In [14]:
X_Hist=root_mean_squared_error(housing_labels,Hist_prediction)
X_Hist

38543.1872877148

In [15]:
Hist_val=-cross_val_score(Hist_Reg,housing,housing_labels,scoring="neg_root_mean_squared_error",cv=10)
pd.Series(Hist_val).describe()

count       10.000000
mean     45801.896871
std        952.108604
min      44636.254610
25%      45069.842120
50%      45626.369159
75%      46180.804579
max      47849.756961
dtype: float64

We will be writing a cell of code without runnning it since it takes quite some time. It is based on decision trees and Gride search CV.

In [16]:
from sklearn.ensemble import RandomForestRegressor
from joblib import Memory
from sklearn.model_selection import GridSearchCV
memory_=Memory(location="cache_dir",verbose=0)
Forest=RandomForestRegressor(random_state=42)
Pipeline_=Pipeline([("Preprocessing",preprocessing),("Random_Forest",Forest)],memory=memory_)
Values_of_parameters=[{"Preprocessing__geo__n_clusters":[1,2,3],"Random_Forest__max_features":[3,6,7]},
                      {"Preprocessing__geo__n_clusters":[5,7],"Random_Forest__max_features":[4,5]}]
grid_search=GridSearchCV(Pipeline_,Values_of_parameters,scoring="neg_root_mean_squared_error",cv=3)
grid_search.fit(housing,housing_labels)
grid_search.best_params_

{'Preprocessing__geo__n_clusters': 7, 'Random_Forest__max_features': 5}

The following code will be to display the results.

In [17]:
cv_results=pd.DataFrame(grid_search.cv_results_)
cv_results.sort_values(by="mean_test_score",ascending=False,inplace=True)
cv_results=cv_results[["param_Preprocessing__geo__n_clusters",
                 "param_Random_Forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
score=["split0","split1","split2","mean_test_score"]
cv_results.columns=["n_clusters","max_features"]+score
cv_results[score]=-cv_results[score].round().astype(np.int64)
cv_results.head()

,n_clusters,max_features,split0,split1,split2,mean_test_score
12,7,5,45854,45743,46657,46085
11,7,4,46183,46019,47213,46472
10,5,5,47419,47502,48454,47792
9,5,4,47807,47297,48744,47949
7,3,6,48483,48825,49959,49089


Now, we will evaluate our data using randomized search CV.

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
Pipeline_=Pipeline([("prep",preprocessing),("Forest",Forest)],memory=memory_)
Value_of_rnd_parameters=[{"prep__geo__n_clusters":randint(low=3,high=50),"Forest__max_features":randint(low=3,high=100)}]
rnd_model=RandomizedSearchCV(Pipeline_,Value_of_rnd_parameters,n_iter=5,cv=3,scoring="neg_root_mean_squared_error",random_state=42)
rnd_model.fit(housing,housing_labels)
rnd_model.best_params_

{'Forest__max_features': 17, 'prep__geo__n_clusters': 45}

Now, we will display results.

In [26]:
cv_results=pd.DataFrame(grid_search.cv_results_)
cv_results.sort_values(by="mean_test_score",ascending=False,inplace=True)
cv_results=cv_results[["param_Preprocessing__geo__n_clusters",
                 "param_Random_Forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
score=["split0","split1","split2","mean_test_score"]
cv_results.columns=["n_clusters","max_features"]+score
cv_results[score]=-cv_results[score].round().astype(np.int64)
cv_results.head()

,n_clusters,max_features,split0,split1,split2,mean_test_score
12,7,5,45854,45743,46657,46085
11,7,4,46183,46019,47213,46472
10,5,5,47419,47502,48454,47792
9,5,4,47807,47297,48744,47949
7,3,6,48483,48825,49959,49089


Now, we need to test our best model on the test set but before doing anything like that we need to get the best features.

In [27]:
final_model=rnd_model.best_estimator_
X_best=final_model["Forest"].feature_importances_
X_best.round(2)
sorted(zip(X_best,final_model["prep"].get_feature_names_out()),reverse=True)

[(0.27069151303879463, 'log__median_income'),
 (0.10302406438757991, 'category__ocean_proximity_INLAND'),
 (0.07151862973114836, 'People_per_house__ratio'),
 (0.06063538156291508, 'bedrooms__ratio'),
 (0.04497045080421451, 'rooms_per_house__ratio'),
 (0.04392663662056182, 'geo__Clusters 3 similarity'),
 (0.03527241050045786, 'geo__Clusters 22 similarity'),
 (0.023629352894128798, 'geo__Clusters 32 similarity'),
 (0.0159035811352478, 'geo__Clusters 17 similarity'),
 (0.01586229046170869, 'remainder__housing_median_age'),
 (0.013028554909590033, 'geo__Clusters 6 similarity'),
 (0.01272104198494122, 'geo__Clusters 18 similarity'),
 (0.010470332179492345, 'geo__Clusters 2 similarity'),
 (0.00997746579480139, 'geo__Clusters 43 similarity'),
 (0.009782698845801541, 'geo__Clusters 38 similarity'),
 (0.009731759161756094, 'geo__Clusters 7 similarity'),
 (0.00959313813603544, 'geo__Clusters 31 similarity'),
 (0.009488818326423327, 'geo__Clusters 21 similarity'),
 (0.00895452820392821, 'geo__Clu

Now, time for testing on our test set.

In [29]:
X_predict=test_set.drop("median_house_value",axis=1)
X_label=test_set["median_house_value"].copy()
X_prediction=final_model.predict(X_predict)
final_error=root_mean_squared_error(X_label,X_prediction)
print(final_error)

42383.42401930013


In [47]:
from scipy import stats
def rmse(squared_error):
    return np.sqrt(np.mean(squared_error))
confidence_level=.95
squared_error=(X_prediction-X_label)**2
boot_=stats.bootstrap([squared_error],rmse,confidence_level=confidence_level,random_state=42)
rmse_lower_bound,rmse_upper_bound=boot_.confidence_interval
print(rmse_lower_bound,rmse_upper_bound)

40413.228078277454 44645.627774962806


Now, saving this model

In [44]:
import joblib
joblib.dump(final_model,"MY_FIRST_PROPER_ALGORITHM.pkl")

PicklingError: Can't pickle <function ratio at 0x000002626383FF60>: it's not the same object as __main__.ratio

In order to call this model back just load this file and make sure you are redefining those custom classes & functions.

In [39]:
# import joblib

# # extra code – excluded for conciseness
# from sklearn.cluster import KMeans
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.metrics.pairwise import rbf_kernel

# def ratio(X):
#     return X[:,[0]]/X[:,[1]]
# def ratio_name(function_Transformer,feature_names_in):
#     return ["ratio"]
# def ratio_pipeline():
#     return make_pipeline(SimpleImputer(strategy="median"),FunctionTransformer(ratio,feature_names_out=ratio_name,),StandardScaler())

# class Cluster(BaseEstimator,TransformerMixin):
#     def __init__(self,random_state=None,gamma=1,n_clusters=10):
#         self.n_clusters=n_clusters
#         self.gamma=gamma
#         self.random_state=random_state
    
#     def fit(self,X,y=None,sample_weight=None):
#         self.K=KMeans(self.n_clusters,n_init=10,random_state=self.random_state)
#         self.K.fit(X,sample_weight=sample_weight)
#         return self
    
#     def transform(self,X):
#         return rbf_kernel(X,self.K.cluster_centers_,gamma=self.gamma)
#     def get_feature_names_out(self,names=None):
#         return [f"Clusters {i} similarity" for i in range (self.n_clusters)]

# final_model_reloaded = joblib.load("MY_FIRST_PROPER_ALGORITHM.pkl")

# new_data = housing.iloc[:5]  # pretend these are new districts
# predictions = final_model_reloaded.predict(new_data)
# predictions

Now, we will be dealing with exercises.

In [ ]:
from sklearn.svm import SVR
model1=SVR()
Pipeline1=Pipeline([("prep",preprocessing),("SVR",model1)])
SVR_parameter_values=[{"SVR__kernel":["linear"],"SVR__C":[3,5,8]},
                      {"SVR__kernel":["rbf"],"SVR__C":[3,6,9],"SVR__gamma":[1e-2, 0.1, 1, 10]}]
grid_s=GridSearchCV(Pipeline1,SVR_parameter_values,scoring="neg_root_mean_squared_error",cv=3)
grid_s.fit(housing.iloc[:5000],housing_labels.iloc[:5000])
grid_s.best_params_

{'SVR__C': 8, 'SVR__kernel': 'linear'}

In [50]:
from sklearn.svm import SVR
from scipy.stats import loguniform
model1=SVR()
Pipeline1=Pipeline([("prep",preprocessing),("SVR",model1)])
SVR_parameter_values=[{"SVR__kernel":["linear"],"SVR__C":randint(low=8,high=1000)},
                      {"SVR__kernel":["rbf"],"SVR__C":randint(low=5,high=1000),"SVR__gamma":loguniform(1e-4,1e1)}]
rand_C=RandomizedSearchCV(Pipeline1,SVR_parameter_values,scoring="neg_root_mean_squared_error",cv=3,n_iter=5)
rand_C.fit(housing.iloc[:5000],housing_labels.iloc[:5000])
rand_C.best_params_

{'SVR__C': 348, 'SVR__kernel': 'linear'}

In [ ]:
grid_SVR_best=grid_s.best_estimator_
Predict_grid=grid_SVR_best.predict(housing.iloc[:5000])
print(root_mean_squared_error(housing_labels.iloc[:5000],Predict_grid))
rand_best=rand_C.best_estimator_
Predict_grid=rand_best.predict(housing.iloc[:5000])
print(root_mean_squared_error(housing_labels.iloc[:5000],Predict_grid))

102817.8169123007
82147.59977782021


In [52]:
from sklearn.feature_selection import SelectFromModel
Pipeline3=Pipeline([("prep",preprocessing),("Model",SelectFromModel(RandomForestRegressor(random_state=42),threshold=0.005)),
                    ("SVR_",SVR(C=rand_C.best_params_["SVR__C"],
                gamma=0.26497040005002437,
                kernel=rand_C.best_params_["SVR__kernel"]))])
Y=cross_val_score(Pipeline3,housing.iloc[:5000],housing_labels.iloc[:5000],cv=3,scoring="neg_root_mean_squared_error")
pd.Series(Y).describe()


count        3.000000
mean    -74669.202860
std       2309.230283
min     -76088.981327
25%     -76001.480113
50%     -75913.978898
75%     -73959.313626
max     -72004.648353
dtype: float64

Creating a custom transformer for K-nearest neighbour.

In [11]:
from sklearn.neighbors import KNeighborsRegressor
class Custom_Transformer(BaseEstimator,TransformerMixin):
    def __init__(self,n_neighbors,weights="uniform"):
        self.n_neighbors=n_neighbors
        self.weights=weights
    def fit(self,X,y):
        self.near=KNeighborsRegressor(n_neighbors=self.n_neighbors,weights=self.weights)
        self.near.fit(X,y)
        return self
    def transform(self,X):
        pred=self.near.predict(X)
        return pred.reshape(-1,1)
knn_transformer = Custom_Transformer(n_neighbors=3, weights="distance")
geo_features = housing[["latitude", "longitude"]]
knn_transformer.fit_transform(geo_features, housing_labels)

array([[486100.66666667],
       [435250.        ],
       [105100.        ],
       ...,
       [148800.        ],
       [500001.        ],
       [234333.33333333]])

How to reimplment the above code so that we can get an output for every estimator

In [12]:
from sklearn.utils.validation import check_array,check_is_fitted
from sklearn.base import clone,MetaEstimatorMixin
class common_estimator(MetaEstimatorMixin,BaseEstimator,TransformerMixin):
    def __init__(self,estimator):
        self.estimator=estimator
    def fit(self,X,Y=None):
        check_array(X)
        self.estimator_=clone(self.estimator) #creates an untrained copy 
        self.estimator_.fit(X,Y) 
        self.n_features_in_=self.estimator_.n_features_in_ #required by sickit learn
        if hasattr(self.estimator_,"feature_names_in_"):
            self.feature_names_in=self.estimator_.feature_names_in_
        return self
    def transform(self,X):
        check_is_fitted(self)
        predictions=self.estimator_.predict(X)
        if predictions.ndim==1:
            return predictions.reshape(-1,1)
        return predictions
    def get_feature_names_out(self,names=None):
        check_is_fitted(self)
        n_outputs=getattr(self.estimator_,"n_outputs_",1)
        estimator_class_name_=self.estimator_.__class__.__name__
        estimator_short_name_=estimator_class_name_.lower().replace("_","")
        return [f"{estimator_short_name_}_predictions_{i}" for i in range(n_outputs)]
knn_reg = KNeighborsRegressor(n_neighbors=3, weights="distance")
knn_transformer = common_estimator(knn_reg)
geo_features = housing[["latitude", "longitude"]]
knn_transformer.fit_transform(geo_features, housing_labels)        
knn_transformer.get_feature_names_out()

['kneighborsregressor_predictions_0']

Question 5: Test some features in RandomizedSearchCV

In [17]:
gossip=[(name,clone(transformer),columns) for name,transformer,columns in preprocessing.transformers]
geo_index=[name for name,_,_ in gossip ].index("geo")
gossip[geo_index]=["geo",knn_transformer,["latitude", "longitude"]]
g=ColumnTransformer(gossip)
g

ColumnTransformer(transformers=[('bedrooms',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('functiontransformer',
                                                  FunctionTransformer(feature_names_out=<function ratio_name at 0x000001FED9730720>,
                                                                      func=<function ratio at 0x000001FED859AFC0>)),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['total_bedrooms', 'total_rooms']),
                                ('People_per_house',
                                 Pi...
                                  'households', 'median_income']),
                                ('category',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001FEDAD1A600>),
                                ['geo',
                                 common_estimator(estimator=KNeighborsRegressor(n_neighbors=3,
                                                                                weights='distance')),
                                 ['latitude', 'longitude']]])

An implementation of standard scaler that I understand.

In [ ]:
class StandardScaler_(BaseEstimator,TransformerMixin):
    def __init__(self,with_mean=True):
        self.with_mean=with_mean
    def fit(self,X,Y=None):
        check_array(X)
        self.n_features_in_ = X.shape[1]
        if self.with_mean:
            self.mean=np.mean(X,axis=0)
        self.std=np.std(X,axis=0,ddof=0)
        self.std[self.std==0]=1
        return self
    def transform(self,X):
        check_is_fitted(X)
        if self.with_mean:
            X=X-self.mean
        return X/self.std
    def inverse_transform(self,X):
        check_is_fitted(X)
        X=X*self.std
        if self.with_mean:
            return X+self.mean
        return X
    def get_feature_names_out(self,input_feature=None):
        if input_feature is None:
            return getattr(self,"feature_names_in_",[f"x{i}" for i in self.n_features_in_] )
        if len(input_feature)!=self.n_features_in_:
            raise ValueError("Invalid number of input features")
        if hasattr(self,"feature_names_in_") and not np.all(
            self.feature_names_in==input_feature
        ):
            raise ValueError("Inputfeature not same as feature_names_in")
        else:
            return input_feature


